In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import re
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

## Code which scrapes the recently reviews (only 100 pages - 1,400 reviews)

In [2]:

def ratebeer_scraper(page_start=1, page_end=2, url_set=None):
    
    beer_name = []
    beer_rating = []
    brewery_name = []
    brewery_location = []
    review_content = []
    reviewer_name = []
 
    if url_set == None:
        print("Need url to run")
        return

    for x in tqdm(range(page_start, page_end + 1)):
        try:
            url_format = url_set.format(x)
            r = requests.get(url_format)
            soup = BeautifulSoup(r.text, 'html.parser')
            table = soup.find('table', attrs={'class': 'table'})
            table_rows = table.find_all('tr')

            for rows in table_rows:

            # Scrape Beer Name    
                try:
                    beer_name.append(rows.find('a', attrs={'style':'font-size:20px; font-weight:bold;'}).text)
                except:
                    beer_name.append(None)
            # Scrape Review Rating  
                try:
                    beer_rating.append(rows.find('span', attrs={'class':'uas'}).text)
                except:
                    beer_rating.append(None)
            # Scrape Brewery Name  
                try:
                    brewery_div = rows.find('div')
                    brewery_name.append(brewery_div.a.text)
                except:
                    brewery_name.append(None)
            # Scrape Brewery Location
                try:
                    brewery_location.append(rows.find('span', attrs={'class':'small'}).text.strip())
                except:
                    brewery_location.append(None)
            # Scrape Reviewer Name
                try:
                    reviewer_name.append(rows.find('span', attrs={'class':'pull-right'}).text.strip())
                except:
                    reviewer_name.append(None)
            # Scrape content
                try:
                    review_content.append(rows.find('div', attrs={'style':"color:#666;"}).text.strip())
                except:
                    review_content.append(None)
        except:
            pass

    beerz = pd.DataFrame({'beer_name': beer_name,
                         'beer_rating': beer_rating,
                        'brewery_name': brewery_name,
                         'brewery_location': brewery_location,
                        'review_content': review_content,
                        'reviewer_name': reviewer_name})
    return beerz



In [4]:
# # Scraping most recent reviews
# url = 'https://www.ratebeer.com/beer-ratings/0/{}/'
# df_recent = ratebeer_scraper(0,100,url)

# Scraping new beers
url = 'https://www.ratebeer.com/beer-ratings/5/{}/'
df_new = ratebeer_scraper(0,1,url)

# Scraping fave beers
url = 'https://www.ratebeer.com/beer-ratings/4/{}/'
df_fave = ratebeer_scraper(0,1,url)

# Scraping rave beers
url = 'https://www.ratebeer.com/beer-ratings/1/{}/'
df_rave = ratebeer_scraper(0,1,url)

# Scraping rant beers
url = 'https://www.ratebeer.com/beer-ratings/2/{}/'
df_rant = ratebeer_scraper(0,1,url)

# Scraping anomalous beers
url = 'https://www.ratebeer.com/beer-ratings/3/{}/'
df_anom = ratebeer_scraper(0,1,url)

In [67]:
# getting rid of non-ascii characters from the reviews as it breaks csv
df_rave.review_content = df_rave.review_content.apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))

In [5]:
# read current data from csv (there is a backup at all_reviews_backup.csv)
# df = pd.read_csv('all_reviews.csv')

In [25]:
# concatenating all of the scrapes together with the old df
df = pd.concat([df, df_recent, df_new, df_fave, df_rave, df_rant, df_anom])
print(len(df))
df.drop_duplicates(inplace=True)
print(len(df))
df.beer_name = df.beer_name.apply(lambda x: x.strip())



14506
12841


In [26]:
df

,beer_name,beer_rating,brewery_location,brewery_name,review_content,reviewer_name
0,St. Peter's Ruby Red Ale,3.3,"Bungay, Suffolk",St. Peter's Brewery,"Ruby color, offwhite coam. Aroma and taste: fl...",alex_leit
1,Shepherd Neame Spitfire (Bottle),2.2,"Faversham, Kent",Shepherd Neame,Disappointing show from the lads at ‘Britain’s...,shivermetimber.gray
2,Arbor Rocketman,3.8,"Bristol, Gloucestershire",Arbor Ales,"Puszka. Mętne o żółtym kolorze. Piana biała,ob...",MarcinG
3,Shepherd Neame India Pale Ale (Bottle),3.4,"Faversham, Kent",Shepherd Neame,Bottle. Amber with bronze color and white foam...,alex_leit
4,Shepherd Neame Christmas Ale,3.3,"Faversham, Kent",Shepherd Neame,Bottle. Aroma: English hops and malts with tof...,alex_leit
...,...,...,...,...,...,...
10,Grimm Mosaic Pop!,1.6,-1.9,Grimm Artisanal Ales,"660ml bottle (Pothouse, Tallinn) (3:2:3:2:6=1....",FatPhil
11,Bone Machine Garden of Death,1.9,-1.6,Bone Machine Brew Co,Can: Poured a mucky orange brown with huge whi...,SHIG
12,Grimm Super You,2.0,-1.5,Grimm Artisanal Ales,660ml bottle - thanks Marduk! (4:2:4:2:8=2.0)\...,FatPhil
13,Magic Rock Dark Arts Hazelnut,1.9,-1.6,Magic Rock Brewing (Lion Co. - Kirin Holdings),"Lovely aromas of sweet chocolate, nuts and van...",Cool_Bond


In [28]:
# df.to_csv('all_reviews.csv',index=False)
# df = pd.read_csv('all_reviews.csv')

## Get more information on all of the beers - Selenium

In [10]:
from selenium import webdriver
import time

In [32]:
# extract already scraped data from previous scrapes so don't rescrape
beer_df = pd.read_csv('beer_deets.csv')

In [33]:
# compare to the unique list to create a new to-be-scraped list
new_beer = df.beer_name[~df.beer_name.isin(beer_df.name_found)].unique()
print("There are", len(new_beer), "new beers to search for!")

There are 1965 new beers to search for!


In [31]:
item_names = []
desc_string = []

driver = webdriver.Chrome(executable_path='/Users/lukebetham/Downloads/chromedriver')

for beer in tqdm(new_beer):
    driver.get("https://www.ratebeer.com/search?beername={}".format(beer))
    time.sleep(2)
    item_tags = driver.find_elements_by_class_name('fg-1')

    x=1
    beer_temp = []
    for item in item_tags:        
        if x <2:
            try:
                item_names.append(item.find_element_by_class_name('fd-r').text)
                x+=1
            except:
                pass
            try:
                beer_temp.append(item.text)
            except:
                pass
    try:
        desc_string.append(beer_temp[4])
    except:
        desc_string.append(None)

In [34]:
beer_deets = pd.DataFrame(desc_string, columns = ['return_string'])
beer_deets

,return_string
0,Flying Dog Raging Bitch Belgian-Style IPA\n🇺🇸 ...
1,Stone R & R Coconut IPA\n🇺🇸 IPA - Flavored • 7...
2,Yaletown Yippie IPA\n🇨🇦 IPA0\n51\n39\n3.32\n(2...
3,Yaletown Downtown Brown\n🇨🇦 Brown Ale0\n25\n23...
4,Yaletown Red Truck Ale\n🇨🇦 Red Ale / Intl Ambe...
...,...
1960,Fürst Carl Franken Ale\n🇩🇪 Blonde Ale / Golden...
1961,Brix City Parrot Jams\n🇺🇸 IIPA DIPA - Hazy / D...
1962,Lone Pine Quantum Cuddle Kitten\n🇺🇸 IIPA DIPA ...
1963,Munich Brew Mafia / Tap House Munich Anniversa...


In [35]:
## cleaning the scraped data
beer_deets.dropna(inplace=True)
beer_deets = beer_deets[~beer_deets.return_string.str.startswith('Logo\n')].copy()
no_alcohol = beer_deets[~beer_deets.return_string.str.contains('%')]
beer_deets = beer_deets[beer_deets.return_string.str.contains('%')].copy()
# maybe create a cant find list so dont keep re-searching?

In [36]:
# sorting the scraped data into columns
beer_deets['name_found'] = beer_deets.return_string.apply(lambda x: x.split('\n')[0].strip())
beer_deets['flag'] = beer_deets.return_string.apply(lambda x: x.split('\n')[1].split('•')[0].split(" ")[0])
beer_deets['type'] = beer_deets.return_string.apply(lambda x: x.split('\n')[1].split('•')[0][2:])
beer_deets['abv'] = beer_deets.return_string.apply(lambda x: float(x.split('\n')[1].split('•')[1].replace("%","")))
beer_deets['rb_overall_score'] = beer_deets.return_string.apply(lambda x: x.split('\n')[2])
beer_deets['rb_style_score'] = beer_deets.return_string.apply(lambda x: x.split('\n')[3])
beer_deets['rb_user_rating'] = beer_deets.return_string.apply(lambda x: x.split('\n')[4])
beer_deets['rb_rating_number'] = beer_deets.return_string.apply(lambda x: x.split('\n')[5].replace("(","").replace(")",""))


In [37]:
# sorting the no-low alcohol
no_alcohol

,return_string
2,Yaletown Yippie IPA\n🇨🇦 IPA0\n51\n39\n3.32\n(2...
3,Yaletown Downtown Brown\n🇨🇦 Brown Ale0\n25\n23...
4,Yaletown Red Truck Ale\n🇨🇦 Red Ale / Intl Ambe...
5,Yaletown Hills Special Wheat\n🇨🇦 Weissbier - H...
9,Nightmare Gunga Rao\n🇺🇸 IPA - Sour / Wild\n-\n...
...,...
1818,De La Vega's Pecan Grill Peanut Butter Pecan P...
1852,Rainbow Olut Alkoholiton Lager\n🇧🇪 Low / No Al...
1871,Cycle Sunday - 2020\n🇺🇸 BarleyWine / WheatWine...
1894,Pyramid Subliminal State Hazy Pale Ale\n🇺🇸 Pal...


In [38]:
beer_deets

,return_string,name_found,flag,type,abv,rb_overall_score,rb_style_score,rb_user_rating,rb_rating_number
0,Flying Dog Raging Bitch Belgian-Style IPA\n🇺🇸 ...,Flying Dog Raging Bitch Belgian-Style IPA,🇺🇸,IPA - Belgian / Saison,8.3,Available from £3.14,96,97,3.79
1,Stone R & R Coconut IPA\n🇺🇸 IPA - Flavored • 7...,Stone R & R Coconut IPA,🇺🇸,IPA - Flavored,7.7,93,93,3.61,527
6,Casey Family Preserves: Anglo Peach & Fantasia...,Casey Family Preserves: Anglo Peach & Fantasia...,🇺🇸,Sour / Wild Beer,6.0,-,-,4.30,1
7,Hanscraft & Co. Backbone Splitter\n🇩🇪 IPA • 6....,Hanscraft & Co. Backbone Splitter,🇩🇪,IPA,6.6,96,97,3.81,402
8,Trillium Fort Point Pale Ale\n🇺🇸 Pale Ale - Am...,Trillium Fort Point Pale Ale,🇺🇸,Pale Ale - American / APA,6.6,100,100,4.08,270
...,...,...,...,...,...,...,...,...,...
1960,Fürst Carl Franken Ale\n🇩🇪 Blonde Ale / Golden...,Fürst Carl Franken Ale,🇩🇪,Blonde Ale / Golden Ale,5.3,25,37,3.01,15
1961,Brix City Parrot Jams\n🇺🇸 IIPA DIPA - Hazy / D...,Brix City Parrot Jams,🇺🇸,IIPA DIPA - Hazy / Double NEIPA,8.0,-,-,3.60,2
1962,Lone Pine Quantum Cuddle Kitten\n🇺🇸 IIPA DIPA ...,Lone Pine Quantum Cuddle Kitten,🇺🇸,IIPA DIPA - Imperial / Double IPA,7.8,-,-,3.80,4
1963,Munich Brew Mafia / Tap House Munich Anniversa...,Munich Brew Mafia / Tap House Munich Anniversator,🇩🇪,Bock - Doppelbock,8.0,80,84,3.49,32


In [22]:
# beer_df.to_csv('beer_deets.csv',index=False)